In [1]:
import json
import os

import contextily as ctx
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import seaborn as sn
from shapely.geometry import Point, LineString
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm

tqdm.pandas()

In [2]:
pd.options.display.max_columns = None

# Data loading

### Kornel

In [96]:
# Kornel
data = pd.read_excel('../../data/raw/Ankiety.xlsx', "Wrocław_ankiety+podróże", engine='openpyxl', header=[0,1])
data.columns = [f'{i}_{j}' for i, j in data.columns]
data = data.iloc[0:14241]

### Marcel

In [3]:
# Marcel
kbr_data = '../../data/raw/KBR/'
kbr_dir = '1_Ankiety_w_gospodarstwach_domowych/'
kbr_file = 'Etap_V_1_1_Ankiety_w_gospodarstwach_domowych_Wroclaw.xlsx'
excel_sheet = 'Wrocław_ankiety+podróże'

data = pd.read_excel(os.path.join(kbr_data, kbr_dir, kbr_file), excel_sheet, header=[0,1], engine='openpyxl')

data.columns = [f'{i}_{j}' for i, j in data.columns]
data = data.iloc[0:14241]

if data.shape[1] == len(set(list(data.columns))):
    print('All column names are unique.')
else:
    print('All column names are NOT unique.')

All column names are unique.


In [4]:
kr_processed_data = '../../data/processed/kr'
kr_processed_file = 'distance_between_regions.json'

with open(os.path.join(kr_processed_data, kr_processed_file), 'r') as f:
    distance_between_regions = json.load(f)

In [5]:
data.head()

,Lp_Lp,IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza),IDENTYFIKACJA ANKIETY_Nr ankietera,IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Godzina wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Nr rejonu komunikacyjnego,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Wprowadzenie opłat za wjazd do centrum dla wszystkich pojazdów,"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dopuszczenie dojazdu do centrum dla pojazdów, które nie zanieczyszczają środowiska (np. pojazdy elektryczne, hybrydowe lub spełniające najwyższe normy spalania)","PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwiązań wspierających kursowanie tramwajów i autobusów, takich jak priorytet przejazdu na skrzyżowaniach oraz wydzielone pasy na jezdniach?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Z jakiego środka transportu zbiorowego korzysta Pan/Pani najczęściej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? \n[tramwaj/autobus],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? [Inne]\n[tramwaj/autobus],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwoju systemu kolei aglomeracyjnej na terenie Wrocławia i wokół niego (rozbudowa układu torowego, zwiększenie częstotliwości itd.)?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia czystość taboru komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niekorzystne ustawienia sygnalizacji świetlnej],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zły stan nawierzchni chodników],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zastawianie chodników przez parkujące samochody],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niebezpieczne zachowania kierowców],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zagrożenie ze strony rowerzystów poruszających się chodnikami],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zbyt wysokie krawężniki],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak bieżącego utrzymania czystości/odśnieżania],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niewystarczająca liczba przejść dla pieszych],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wska

In [6]:
chosen_cols = [
    # 'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Z jakiego środka transportu zbiorowego korzysta Pan/Pani\xa0najczęściej?',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?\xa0',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia\xa0punktualność\xa0komunikacji zbiorowej\xa0we Wrocławiu?\xa0',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i\xa0parkingów rowerowych)?',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niekorzystne ustawienia sygnalizacji świetlnej]',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką]',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zły stan nawierzchni chodników]',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zastawianie chodników przez parkujące samochody]',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niebezpieczne zachowania kierowców]',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zagrożenie ze strony rowerzystów poruszających się chodnikami]',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zbyt wysokie krawężniki]',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak bieżącego utrzymania czystości/odśnieżania]',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niewystarczająca liczba przejść dla pieszych]',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak miejsc wypoczynku na trasie dojścia (np. ławki, zieleń)]',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [uciążliwy ruch kołowy]',
    'PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niewłaściwe oświetlenie ciągów pieszych]',
    'DANE O GOSPODARSTWIE DOMOWYM_Liczba osób w gospodarstwie domowym [ogółem]',
    'DANE O GOSPODARSTWIE DOMOWYM_Liczba osób w gospodarstwie domowym [powyżej 6 roku życia]',
    'DANE O GOSPODARSTWIE DOMOWYM_Liczba środków transportu w gospodarstwie domowym [Samochód prywatny, zarejestrowany na osobę z gosp. domowego]',
    'DANE O GOSPODARSTWIE DOMOWYM_Liczba środków transportu w gospodarstwie domowym [Samochód prywatny, nie zarejestrowany na osobę z gosp. domowego [użyczone]]',
    'DANE O GOSPODARSTWIE DOMOWYM_Liczba środków transportu w gospodarstwie domowym [Samochód służbowy]',
    'DANE O GOSPODARSTWIE DOMOWYM_Liczba środków transportu w gospodarstwie domowym [Rower]',
    'DANE O RESPONDENCIE_Przedział wiekowy',
    'DANE O RESPONDENCIE_Płeć',
    # 'DANE O RESPONDENCIE_Zajęcie podstawowe',
    'DANE O RESPONDENCIE_Kondycja fizyczna',
    'DANE O RESPONDENCIE_Opieka nad innymi osobami',
    'DANE O RESPONDENCIE_Posiadanie prawa jazdy kat. B',
    'DANE O RESPONDENCIE_Posiadanie biletu okresowego',
    'DANE O RESPONDENCIE_Posiadanie ulgi na przejazd komunikacją zbiorową we Wrocławiu',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Godzina rozpoczęcia podróży\n[gg:mm:ss]',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Z jakiego miejsca',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Nr rejonu',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Nr rejonu',
    '23:59:00_pora dnia',
    '23:59:00_motywacje',
    '23:59:00_środek transportu grupa'
]

data = data[chosen_cols]

data.columns = [
    # 'Z jakiego środka transportu zbiorowego korzysta Pan/Pani najczęściej?',
    'Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?',
    'Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?',
    'Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?',
    'PIESZO Niekorzystne ustawienia sygnalizacji świetlnej',
    'PIESZO Brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką',
    'PIESZO Zły stan nawierzchni chodników',
    'PIESZO Zastawianie chodników przez parkujące samochody',
    'PIESZO Niebezpieczne zachowania kierowców',
    'PIESZO Zagrożenie ze strony rowerzystów poruszających się chodnikami',
    'PIESZO Zbyt wysokie krawężniki',
    'PIESZO Brak bieżącego utrzymania czystości/odśnieżania',
    'PIESZO Niewystarczająca liczba przejść dla pieszych',
    'PIESZO Brak miejsc wypoczynku na trasie dojścia (np. ławki, zieleń)',
    'PIESZO Uciążliwy ruch kołowy',
    'PIESZO Niewłaściwe oświetlenie ciągów pieszych',
    'Liczba osób w gospodarstwie domowym [ogółem]',
    'Liczba osób w gospodarstwie domowym [powyżej 6 roku życia]',
    'Samochód prywatny, zarejestrowany na osobę z gosp. domowego',
    'Samochód prywatny, nie zarejestrowany na osobę z gosp. domowego [użyczone]',
    'Samochód służbowy',
    'Rower',
    'Przedział wiekowy',
    'Płeć',
    # 'Zajęcie podstawowe',
    'Kondycja fizyczna',
    'Opieka nad innymi osobami',
    'Posiadanie prawa jazdy kat. B',
    'Posiadanie biletu okresowego',
    'Posiadanie ulgi na przejazd komunikację zbiorową',
    'ŹRÓDŁO Czas rozpoczęcia',
    'ŹRÓDŁO Z jakiego miejsca',
    'ŹRÓDŁO Nr rejonu',
    'CEL Do jakiego miejsca',
    'CEL Nr rejonu',
    'Pora dnia (godzina)',
    'Motywacje (skąd-dokąd)',
    'Środek transportu grupa'
]

In [7]:
data.head()

,Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,PIESZO Niekorzystne ustawienia sygnalizacji świetlnej,PIESZO Brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką,PIESZO Zły stan nawierzchni chodników,PIESZO Zastawianie chodników przez parkujące samochody,PIESZO Niebezpieczne zachowania kierowców,PIESZO Zagrożenie ze strony rowerzystów poruszających się chodnikami,PIESZO Zbyt wysokie krawężniki,PIESZO Brak bieżącego utrzymania czystości/odśnieżania,PIESZO Niewystarczająca liczba przejść dla pieszych,"PIESZO Brak miejsc wypoczynku na trasie dojścia (np. ławki, zieleń)",PIESZO Uciążliwy ruch kołowy,PIESZO Niewłaściwe oświetlenie ciągów pieszych,Liczba osób w gospodarstwie domowym [ogółem],Liczba osób w gospodarstwie domowym [powyżej 6 roku życia],"Samochód prywatny, zarejestrowany na osobę z gosp. domowego","Samochód prywatny, nie zarejestrowany na osobę z gosp. domowego [użyczone]",Samochód służbowy,Rower,Przedział wiekowy,Płeć,Kondycja fizyczna,Opieka nad innymi osobami,Posiadanie prawa jazdy kat. B,Posiadanie biletu okresowego,Posiadanie ulgi na przejazd komunikację zbiorową,ŹRÓDŁO Czas rozpoczęcia,ŹRÓDŁO Z jakiego miejsca,ŹRÓDŁO Nr rejonu,CEL Do jakiego miejsca,CEL Nr rejonu,Pora dnia (godzina),Motywacje (skąd-dokąd),Środek transportu grupa
0,bardzo dobrze,raczej dobrze,bardzo dobrze,Nie,Nie,Nie,Nie,Nie,Tak,Tak,Nie,Nie,Nie,Tak,Nie,1.0,1.0,0.0,0.0,0.0,1.0,61 i więcej (emeryci kobiety),Kobieta,dobry stan zdrowia,nie dotyczy,Tak,Tak,0.5,09:00:00,dom,67.0,Inne,68.0,9:00-9:59,dom-Inne,komunikacja samochodowa
1,bardzo dobrze,raczej dobrze,bardzo dobrze,Nie,Nie,Nie,Nie,Nie,Tak,Tak,Nie,Nie,Nie,Tak,Nie,1.0,1.0,0.0,0.0,0.0,1.0,61 i więcej (emeryci kobiety),Kobieta,dobry stan zdrowia,nie dotyczy,Tak,Tak,0.5,09:40:00,Inne,68.0,dom,67.0,9:00-9:59,Inne-dom,komunikacja samochodowa
2,bardzo dobrze,raczej dobrze,raczej dobrze,Nie,Nie,Nie,Tak,Nie,Nie,Nie,Nie,Tak,Nie,Nie,Nie,2.0,2.0,0.0,1.0,0.0,1.0,20-24 (wiek studencki),Kobieta,dobry stan zdrowia,nie dotyczy,Tak,Tak,0.5,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN
3,bardzo dobrze,raczej dobrze,raczej dobrze,Nie,Nie,Nie,Tak,Nie,Nie,Nie,Nie,Tak,Nie,Nie,Nie,2.0,2.0,0.0,1.0,0.0,1.0,20-24 (wiek studencki),Mężczyzna,dobry stan zdrowia,nie dotyczy,Tak,Tak,0.5,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN
4,bardzo dobrze,bardzo dobrze,ani dobrze ani źle,Nie,Tak,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,2.0,2.0,1.0,0.0,0.0,0.0,61 i więcej (emeryci kobiety),Kobieta,dolegliwości zdrowotne,nie dotyczy,Nie,Nie,0.5,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN


In [8]:
data.shape

(14241, 36)

# Data processing

## Drop rows with NaN values and prepare column 'Samochód'

In [9]:
data = data.dropna()

data['Samochód'] = data['Samochód prywatny, zarejestrowany na osobę z gosp. domowego'] + data['Samochód prywatny, nie zarejestrowany na osobę z gosp. domowego [użyczone]'] + data['Samochód służbowy']

data = data.drop(columns=['Samochód prywatny, zarejestrowany na osobę z gosp. domowego', 'Samochód prywatny, nie zarejestrowany na osobę z gosp. domowego [użyczone]', 'Samochód służbowy'])

data.shape

(10626, 34)

## Add column with the dsitance of the trip

In [10]:
regions_unique = distance_between_regions.keys()

data = data[
    (data['ŹRÓDŁO Nr rejonu'].astype('int').astype('str').isin(regions_unique)) &
    (data['CEL Nr rejonu'].astype('int').astype('str').isin(regions_unique))
]

In [11]:
def add_regions_num(row):
    row['Przebyta odległość'] = distance_between_regions[str(int(row['ŹRÓDŁO Nr rejonu']))][str(int(row['CEL Nr rejonu']))]

    return row


data = data.progress_apply(add_regions_num, axis=1)

100%|██████████| 10138/10138 [00:09<00:00, 1120.16it/s]


## Add column with age_sex_comb

In [12]:
data['age_bracket'] = data['Przedział wiekowy']
data['sex'] = data['Płeć']

In [13]:
age_bracket_map_dict = {
    '6-15 (dzieci)': '6-15',
    '16-19 (młodzież)': '16-19',
    '20-24 (wiek studencki)': '20-24',
    '25-44 (młodsi pracownicy)': '25-44',
    '45-60 (starsi pracownicy kobiety)': '45-60',  # '45-60(K)',
    '45-65 (starsi pracownicy mężczyźni)': '45-65',  # '45-65(M)',
    '61 i więcej (emeryci kobiety)': '61-x',  # '61-(K)',
    '66 i więcej (emeryci mężczyźni)': '66-x'  # '66-(M)'
}
data.replace({'age_bracket': age_bracket_map_dict}, inplace=True)

sex_map_dict = {
    'Kobieta': 'K',
    'Mężczyzna': 'M'
}
data.replace({'sex': sex_map_dict}, inplace=True)

data['age_sex_comb'] = data['age_bracket'] + '_' + data['sex']

In [14]:
data = data.drop(columns=['age_bracket', 'sex'], axis=1)

In [15]:
data_copy = data

In [16]:
data.head()

,Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,PIESZO Niekorzystne ustawienia sygnalizacji świetlnej,PIESZO Brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką,PIESZO Zły stan nawierzchni chodników,PIESZO Zastawianie chodników przez parkujące samochody,PIESZO Niebezpieczne zachowania kierowców,PIESZO Zagrożenie ze strony rowerzystów poruszających się chodnikami,PIESZO Zbyt wysokie krawężniki,PIESZO Brak bieżącego utrzymania czystości/odśnieżania,PIESZO Niewystarczająca liczba przejść dla pieszych,"PIESZO Brak miejsc wypoczynku na trasie dojścia (np. ławki, zieleń)",PIESZO Uciążliwy ruch kołowy,PIESZO Niewłaściwe oświetlenie ciągów pieszych,Liczba osób w gospodarstwie domowym [ogółem],Liczba osób w gospodarstwie domowym [powyżej 6 roku życia],Rower,Przedział wiekowy,Płeć,Kondycja fizyczna,Opieka nad innymi osobami,Posiadanie prawa jazdy kat. B,Posiadanie biletu okresowego,Posiadanie ulgi na przejazd komunikację zbiorową,ŹRÓDŁO Czas rozpoczęcia,ŹRÓDŁO Z jakiego miejsca,ŹRÓDŁO Nr rejonu,CEL Do jakiego miejsca,CEL Nr rejonu,Pora dnia (godzina),Motywacje (skąd-dokąd),Środek transportu grupa,Samochód,Przebyta odległość,age_sex_comb
0,bardzo dobrze,raczej dobrze,bardzo dobrze,Nie,Nie,Nie,Nie,Nie,Tak,Tak,Nie,Nie,Nie,Tak,Nie,1.0,1.0,1.0,61 i więcej (emeryci kobiety),Kobieta,dobry stan zdrowia,nie dotyczy,Tak,Tak,0.5,09:00:00,dom,67.0,Inne,68.0,9:00-9:59,dom-Inne,komunikacja samochodowa,0.0,1223.212986,61-x_K
1,bardzo dobrze,raczej dobrze,bardzo dobrze,Nie,Nie,Nie,Nie,Nie,Tak,Tak,Nie,Nie,Nie,Tak,Nie,1.0,1.0,1.0,61 i więcej (emeryci kobiety),Kobieta,dobry stan zdrowia,nie dotyczy,Tak,Tak,0.5,09:40:00,Inne,68.0,dom,67.0,9:00-9:59,Inne-dom,komunikacja samochodowa,0.0,1223.212986,61-x_K
5,bardzo dobrze,bardzo dobrze,bardzo dobrze,Nie,Tak,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,2.0,2.0,0.0,45-65 (starsi pracownicy mężczyźni),Mężczyzna,dobry stan zdrowia,nie dotyczy,Tak,Nie,0.5,09:00:00,dom,67.0,Inne,68.0,9:00-9:59,dom-Inne,komunikacja samochodowa,1.0,1223.212986,45-65_M
6,bardzo dobrze,bardzo dobrze,bardzo dobrze,Nie,Tak,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,2.0,2.0,0.0,45-65 (starsi pracownicy mężczyźni),Mężczyzna,dobry stan zdrowia,nie dotyczy,Tak,Nie,0.5,09:40:00,Inne,68.0,dom,67.0,9:00-9:59,Inne-dom,komunikacja samochodowa,1.0,1223.212986,45-65_M
7,bardzo dobrze,raczej dobrze,nie korzystam z systemu dróg i parkingów rower...,Nie,Nie,Nie,Nie,Tak,Nie,Nie,Nie,Nie,Nie,Nie,Nie,1.0,1.0,0.0,25-44 (młodsi pracownicy),Kobieta,dobry stan zdrowia,nie dotyczy,Tak,Tak,0.5,18:00:00,dom,67.0,Inne,29.0,18:00-18:59,dom-Inne,komunikacja zbiorowa,0.0,2637.323757,25-44_K


## Features dataframe processing - map string values to numerical

### 'Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?'

In [17]:
data['Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?'].unique()

array(['bardzo dobrze', 'raczej dobrze',
       'nie korzystam z komunikacji zbiorowej', 'raczej źle',
       'ani dobrze ani źle', 'bardzo źle'], dtype=object)

In [18]:
var_map_dict = {
    'bardzo źle': 0,
    'raczej źle': 1,
    'ani dobrze ani źle': 2,
    'nie korzystam z komunikacji zbiorowej': 2,
    'raczej dobrze': 3,
    'bardzo dobrze': 4
}

data.replace({'Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?': var_map_dict}, inplace=True)

### 'Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?'

In [19]:
data['Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?'].unique()

array(['raczej dobrze', 'bardzo dobrze', 'raczej źle',
       'ani dobrze ani źle', 'nie korzystam z komunikacji zbiorowej',
       'bardzo źle'], dtype=object)

In [20]:
var_map_dict = {
    'bardzo źle': 0,
    'raczej źle': 1,
    'ani dobrze ani źle': 2,
    'nie korzystam z komunikacji zbiorowej': 2,
    'raczej dobrze': 3,
    'bardzo dobrze': 4
}

data.replace({'Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?': var_map_dict}, inplace=True)

### 'Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?'

In [21]:
data['Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?'].unique()

array(['bardzo dobrze',
       'nie korzystam z systemu dróg i parkingów rowerowych',
       'raczej źle', 'raczej dobrze', 'ani dobrze ani źle', 'bardzo źle',
       'nie korzystam z komunikacji zbiorowej'], dtype=object)

In [22]:
var_map_dict = {
    'bardzo źle': 0,
    'raczej źle': 1,
    'ani dobrze ani źle': 2,
    'nie korzystam z komunikacji zbiorowej': 2,
    'nie korzystam z systemu dróg i parkingów rowerowych': 2,
    'raczej dobrze': 3,
    'bardzo dobrze': 4
}

data.replace({'Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?': var_map_dict}, inplace=True)

### 'PIESZO Niekorzystne ustawienia sygnalizacji świetlnej'

In [23]:
data['PIESZO Niekorzystne ustawienia sygnalizacji świetlnej'].unique()

array(['Nie', 'Tak', 'nie', 'tak'], dtype=object)

In [24]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'PIESZO Niekorzystne ustawienia sygnalizacji świetlnej': var_map_dict}, inplace=True)

### 'PIESZO Brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką'

In [25]:
data['PIESZO Brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką'].unique()

array(['Nie', 'Tak', 'nie', 'tak'], dtype=object)

In [26]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'PIESZO Brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką': var_map_dict}, inplace=True)

### 'PIESZO Zły stan nawierzchni chodników'

In [27]:
data['PIESZO Zły stan nawierzchni chodników'].unique()

array(['Nie', 'Tak', 'nie', 'tak'], dtype=object)

In [28]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'PIESZO Zły stan nawierzchni chodników': var_map_dict}, inplace=True)

### PIESZO Zastawianie chodników przez parkujące samochody

In [29]:
data['PIESZO Zastawianie chodników przez parkujące samochody'].unique()

array(['Nie', 'Tak', 'tak', 'nie'], dtype=object)

In [30]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'PIESZO Zastawianie chodników przez parkujące samochody': var_map_dict}, inplace=True)

### PIESZO Niebezpieczne zachowania kierowców

In [31]:
data['PIESZO Niebezpieczne zachowania kierowców'].unique()

array(['Nie', 'Tak', 'nie', 'tak'], dtype=object)

In [32]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'PIESZO Niebezpieczne zachowania kierowców': var_map_dict}, inplace=True)

### 'PIESZO Zagrożenie ze strony rowerzystów poruszających się chodnikami'

In [33]:
data['PIESZO Zagrożenie ze strony rowerzystów poruszających się chodnikami'].unique()

array(['Tak', 'Nie', 'nie', 'tak'], dtype=object)

In [34]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'PIESZO Zagrożenie ze strony rowerzystów poruszających się chodnikami': var_map_dict}, inplace=True)

### 'PIESZO Zbyt wysokie krawężniki'

In [35]:
data['PIESZO Zbyt wysokie krawężniki'].unique()

array(['Tak', 'Nie', 'nie', 'tak'], dtype=object)

In [36]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'PIESZO Zbyt wysokie krawężniki': var_map_dict}, inplace=True)

### 'PIESZO Brak bieżącego utrzymania czystości/odśnieżania'

In [37]:
data['PIESZO Brak bieżącego utrzymania czystości/odśnieżania'].unique()

array(['Nie', 'Tak', 'tak', 'nie'], dtype=object)

In [38]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'PIESZO Brak bieżącego utrzymania czystości/odśnieżania': var_map_dict}, inplace=True)

### 'PIESZO Niewystarczająca liczba przejść dla pieszych'

In [39]:
data['PIESZO Niewystarczająca liczba przejść dla pieszych'].unique()

array(['Nie', 'Tak', 'nie', 'tak'], dtype=object)

In [40]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'PIESZO Niewystarczająca liczba przejść dla pieszych': var_map_dict}, inplace=True)

### 'PIESZO Brak miejsc wypoczynku na trasie dojścia (np. ławki, zieleń)'

In [41]:
data['PIESZO Brak miejsc wypoczynku na trasie dojścia (np. ławki, zieleń)'].unique()

array(['Nie', 'Tak', 'nie', 'tak'], dtype=object)

In [42]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'PIESZO Brak miejsc wypoczynku na trasie dojścia (np. ławki, zieleń)': var_map_dict}, inplace=True)

### 'PIESZO Uciążliwy ruch kołowy'

In [43]:
data['PIESZO Uciążliwy ruch kołowy'].unique()

array(['Tak', 'Nie', 'nie', 'tak'], dtype=object)

In [44]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'PIESZO Uciążliwy ruch kołowy': var_map_dict}, inplace=True)

### 'PIESZO Niewłaściwe oświetlenie ciągów pieszych'

In [45]:
data['PIESZO Niewłaściwe oświetlenie ciągów pieszych'].unique()

array(['Nie', 'Tak', 'nie', 'tak'], dtype=object)

In [46]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'PIESZO Niewłaściwe oświetlenie ciągów pieszych': var_map_dict}, inplace=True)

### 'Przedział wiekowy'

In [47]:
data['Przedział wiekowy'].unique()

array(['61 i więcej (emeryci kobiety)',
       '45-65 (starsi pracownicy mężczyźni)', '25-44 (młodsi pracownicy)',
       '66 i więcej (emeryci mężczyźni)',
       '45-60 (starsi pracownicy kobiety)', '20-24 (wiek studencki)',
       '16-19 (młodzież)', '6-15 (dzieci)'], dtype=object)

In [48]:
var_map_dict = {
    '6-15 (dzieci)': 0,
    '16-19 (młodzież)': 1,
    '20-24 (wiek studencki)': 2,
    '25-44 (młodsi pracownicy)': 3,
    '45-60 (starsi pracownicy kobiety)': 4,
    '45-65 (starsi pracownicy mężczyźni)': 4,
    '61 i więcej (emeryci kobiety)': 5,
    '66 i więcej (emeryci mężczyźni)': 5
}

data.replace({'Przedział wiekowy': var_map_dict}, inplace=True)

### 'Płeć'

In [49]:
data['Płeć'].unique()

array(['Kobieta', 'Mężczyzna'], dtype=object)

In [50]:
var_map_dict = {
    'Mężczyzna': 0,
    'Kobieta': 1
}

data.replace({'Płeć': var_map_dict}, inplace=True)

### 'Kondycja fizyczna'

In [51]:
data['Kondycja fizyczna'].unique()

array(['dobry stan zdrowia',
       'osoba starsza dolegliwości związane z\xa0wiekiem',
       'kłopoty z poruszaniem się', 'dolegliwości zdrowotne', 'Inne',
       'inne'], dtype=object)

In [52]:
var_map_dict = {
    'kłopoty z poruszaniem się': 0,
    'dolegliwości zdrowotne': 0,
    'osoba starsza dolegliwości związane z\xa0wiekiem': 0,
    'Inne': 0,
    'inne': 0,
    'dobry stan zdrowia': 1
}

data.replace({'Kondycja fizyczna': var_map_dict}, inplace=True)

### 'Opieka nad innymi osobami'

In [53]:
data['Opieka nad innymi osobami'].unique()

array(['nie dotyczy', 'opieka nad dzieckiem',
       'opieka nad osobą w podeszłym wieku', 'opieka nad inną osobą'],
      dtype=object)

In [54]:
var_map_dict = {
    'nie dotyczy': 0,
    'opieka nad dzieckiem': 1,
    'opieka nad osobą w podeszłym wieku': 1,
    'opieka nad inną osobą': 1
}

data.replace({'Opieka nad innymi osobami': var_map_dict}, inplace=True)

### 'Posiadanie prawa jazdy kat. B'

In [55]:
data['Posiadanie prawa jazdy kat. B'].unique()

array(['Tak', 'Nie', 'tak'], dtype=object)

In [56]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'Posiadanie prawa jazdy kat. B': var_map_dict}, inplace=True)

### 'Posiadanie biletu okresowego'

In [57]:
data['Posiadanie biletu okresowego'].unique()

array(['Tak', 'Nie', 'tak'], dtype=object)

In [58]:
var_map_dict = {
    'Nie': 0,
    'nie': 0,
    'Tak': 1,
    'tak': 1
}

data.replace({'Posiadanie biletu okresowego': var_map_dict}, inplace=True)

### 'Posiadanie ulgi na przejazd komunikację zbiorową'

In [59]:
data['Posiadanie ulgi na przejazd komunikację zbiorową'].unique()

array([0.5, 1, 'brak ulgi'], dtype=object)

In [60]:
var_map_dict = {
    'brak ulgi': 0,
    0.5: 0.5,
    1: 1,
}

data.replace({'Posiadanie ulgi na przejazd komunikację zbiorową': var_map_dict}, inplace=True)

### 'Pora dnia (godzina)'

In [61]:
sorted(data['Pora dnia (godzina)'].unique())

['0:00-4:59',
 '10:00-10:59',
 '11:00-11:59',
 '12:00-12:59',
 '13:00-13:59',
 '14:00-14:59',
 '15:00-15:59',
 '16:00-16:59',
 '17:00-17:59',
 '18:00-18:59',
 '19:00-19:59',
 '20:00-23:59',
 '5:00-5:59',
 '6:00-6:59',
 '7:00-7:59',
 '8:00-8:59',
 '9:00-9:59']

In [62]:
var_map_dict = {
    '0:00-4:59': 0,         # noc           0-5
    '5:00-5:59': 1,         # rano          5-10
    '6:00-6:59': 1,         # rano          5-10
    '7:00-7:59': 1,         # rano          5-10
    '8:00-8:59': 1,         # rano          5-10
    '9:00-9:59': 1,         # rano          5-10
    '10:00-10:59': 2,       # południe      10-14
    '11:00-11:59': 2,       # południe      10-14
    '12:00-12:59': 2,       # południe      10-14
    '13:00-13:59': 2,       # południe      10-14
    '14:00-14:59': 3,       # popołudnie    14-19
    '15:00-15:59': 3,       # popołudnie    14-19
    '16:00-16:59': 3,       # popołudnie    14-19
    '17:00-17:59': 3,       # popołudnie    14-19
    '18:00-18:59': 3,       # popołudnie    14-19
    '19:00-19:59': 4,       # wieczór       19-0
    '20:00-23:59': 4        # wieczór       19-0
}

data.replace({'Pora dnia (godzina)': var_map_dict}, inplace=True)

### 'Środek transportu grupa'

In [63]:
data['Środek transportu grupa'].unique()

array(['komunikacja samochodowa', 'komunikacja zbiorowa', 'pieszo',
       'rower', 'inne'], dtype=object)

In [64]:
data['Środek transportu grupa'].value_counts()

komunikacja samochodowa    3605
komunikacja zbiorowa       2976
pieszo                     2752
rower                       742
inne                         63
Name: Środek transportu grupa, dtype: int64

In [65]:
drop_inne = True

if drop_inne:
    data = data[data['Środek transportu grupa'] != 'inne']
    var_map_dict = {
        'komunikacja samochodowa': 0,
        'komunikacja zbiorowa': 1,
        'pieszo': 2,
        'rower': 3,
    }
else:
    var_map_dict = {
        'komunikacja samochodowa': 0,
        'komunikacja zbiorowa': 1,
        'pieszo': 2,
        'rower': 3,
        'inne': 4
    }

data.replace({'Środek transportu grupa': var_map_dict}, inplace=True)

### 'CEL Do jakiego miejsca'

In [66]:
data['CEL Do jakiego miejsca'] = data['CEL Do jakiego miejsca'].str.replace('Inne', 'inne')
sorted(data['CEL Do jakiego miejsca'].unique())

['dom', 'inne', 'praca', 'szkoła', 'uczelnia']

In [67]:
data = data.merge(
    pd.get_dummies(data['CEL Do jakiego miejsca'], prefix='CEL'),
    left_index=True, 
    right_index=True
)

### 'ŹRÓDŁO Czas rozpoczęcia'

In [92]:
data['ŹRÓDŁO Godzina rozpoczęcia'] = data['ŹRÓDŁO Czas rozpoczęcia'].astype('str').apply(lambda x: x.split(':')[0])

In [98]:
data['ŹRÓDŁO Godzina rozpoczęcia'].unique()

array(['09', '18', '19', '16', '12', '13', '06', '14', '05', '07', '08',
       '15', '10', '11', '17', '20', '04', '21', '22', '23', '03', '00',
       '1899-12-30 00'], dtype=object)

In [102]:
data = data[data['ŹRÓDŁO Godzina rozpoczęcia'] != '1899-12-30 00']

In [106]:
data['ŹRÓDŁO Godzina rozpoczęcia'] = data['ŹRÓDŁO Godzina rozpoczęcia'].astype('int')

In [107]:
data['ŹRÓDŁO Godzina rozpoczęcia'].unique()

array([ 9, 18, 19, 16, 12, 13,  6, 14,  5,  7,  8, 15, 10, 11, 17, 20,  4,
       21, 22, 23,  3,  0])

In [111]:
data.head()

,Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,PIESZO Niekorzystne ustawienia sygnalizacji świetlnej,PIESZO Brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką,PIESZO Zły stan nawierzchni chodników,PIESZO Zastawianie chodników przez parkujące samochody,PIESZO Niebezpieczne zachowania kierowców,PIESZO Zagrożenie ze strony rowerzystów poruszających się chodnikami,PIESZO Zbyt wysokie krawężniki,PIESZO Brak bieżącego utrzymania czystości/odśnieżania,PIESZO Niewystarczająca liczba przejść dla pieszych,"PIESZO Brak miejsc wypoczynku na trasie dojścia (np. ławki, zieleń)",PIESZO Uciążliwy ruch kołowy,PIESZO Niewłaściwe oświetlenie ciągów pieszych,Liczba osób w gospodarstwie domowym [ogółem],Liczba osób w gospodarstwie domowym [powyżej 6 roku życia],Rower,Przedział wiekowy,Płeć,Kondycja fizyczna,Opieka nad innymi osobami,Posiadanie prawa jazdy kat. B,Posiadanie biletu okresowego,Posiadanie ulgi na przejazd komunikację zbiorową,ŹRÓDŁO Czas rozpoczęcia,ŹRÓDŁO Z jakiego miejsca,ŹRÓDŁO Nr rejonu,CEL Do jakiego miejsca,CEL Nr rejonu,Pora dnia (godzina),Motywacje (skąd-dokąd),Środek transportu grupa,Samochód,Przebyta odległość,age_sex_comb,CEL_dom,CEL_inne,CEL_praca,CEL_szkoła,CEL_uczelnia,ŹRÓDŁO Godzina rozpoczęcia
0,4,3,4,0,0,0,0,0,1,1,0,0,0,1,0,1.0,1.0,1.0,5,1,1,0,1,1,0.5,09:00:00,dom,67.0,inne,68.0,1,dom-Inne,0,0.0,1223.212986,61-x_K,0,1,0,0,0,9
1,4,3,4,0,0,0,0,0,1,1,0,0,0,1,0,1.0,1.0,1.0,5,1,1,0,1,1,0.5,09:40:00,Inne,68.0,dom,67.0,1,Inne-dom,0,0.0,1223.212986,61-x_K,1,0,0,0,0,9
5,4,4,4,0,1,0,0,0,0,0,0,0,0,0,0,2.0,2.0,0.0,4,0,1,0,1,0,0.5,09:00:00,dom,67.0,inne,68.0,1,dom-Inne,0,1.0,1223.212986,45-65_M,0,1,0,0,0,9
6,4,4,4,0,1,0,0,0,0,0,0,0,0,0,0,2.0,2.0,0.0,4,0,1,0,1,0,0.5,09:40:00,Inne,68.0,dom,67.0,1,Inne-dom,0,1.0,1223.212986,45-65_M,1,0,0,0,0,9
7,4,3,2,0,0,0,0,1,0,0,0,0,0,0,0,1.0,1.0,0.0,3,1,1,0,1,1,0.5,18:00:00,dom,67.0,inne,29.0,3,dom-Inne,1,0.0,2637.323757,25-44_K,0,1,0,0,0,18


## Save data dataframe

In [110]:
out_dir = '../../data/processed/mc/decision_tree'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

out_file = 'data_dist.csv'
out_path = os.path.join(out_dir, out_file)

out_path

'../../data/processed/mc/decission_tree/data_dist.csv'

In [112]:
data.to_csv(out_path)

In [113]:
data = pd.read_csv(out_path, index_col = 0)

# Decision Tree distributions

## Distribution: rating the comfort of public transport grouped by combinations age group + gender

### Prepare data

In [171]:
data_grouped = data.groupby(['age_sex_comb', 'Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?'])['Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?'].count().reset_index(name='count')

data_grouped.shape[0]

58

In [172]:
data_grouped.head()

,age_sex_comb,Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,count
0,16-19_K,0,2
1,16-19_K,1,8
2,16-19_K,2,49
3,16-19_K,3,85
4,16-19_K,4,24


In [173]:
data[
    (data['age_sex_comb'] == '16-19_K') &
    (data['Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?'] == 2)
].shape[0]

49

### Distribution

In [174]:
unique_age_sex_comb = data_grouped['age_sex_comb'].unique()
max_rating = data_grouped['Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?'].max()

pub_trans_comfort_dist = {}

for age_sex_comb in unique_age_sex_comb:
    pub_trans_comfort_dist[age_sex_comb] = {}

    for rating in range(0, max_rating+1):
        try:
            count = int(data_grouped[
                (data_grouped['Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?'] == rating) & 
                (data_grouped['age_sex_comb'] == age_sex_comb)
            ]['count'])
        except TypeError:
            count = 1

        pub_trans_comfort_dist[age_sex_comb][rating] = count

In [175]:
pub_trans_comfort_dist

{'16-19_K': {0: 2, 1: 8, 2: 49, 3: 85, 4: 24},
 '16-19_M': {0: 1, 1: 13, 2: 76, 3: 139, 4: 23},
 '20-24_K': {0: 3, 1: 26, 2: 130, 3: 256, 4: 129},
 '20-24_M': {0: 4, 1: 20, 2: 147, 3: 239, 4: 74},
 '25-44_K': {0: 8, 1: 65, 2: 775, 3: 649, 4: 223},
 '25-44_M': {0: 8, 1: 44, 2: 827, 3: 464, 4: 116},
 '45-60_K': {0: 12, 1: 52, 2: 463, 3: 481, 4: 197},
 '45-65_M': {0: 9, 1: 24, 2: 634, 3: 402, 4: 122},
 '6-15_K': {0: 2, 1: 20, 2: 258, 3: 183, 4: 78},
 '6-15_M': {0: 1, 1: 8, 2: 168, 3: 189, 4: 53},
 '61-x_K': {0: 14, 1: 31, 2: 419, 3: 622, 4: 295},
 '66-x_M': {0: 2, 1: 12, 2: 252, 3: 306, 4: 140}}

### Probability distribution

In [176]:
for age_sex_comb, ratings in pub_trans_comfort_dist.items():
    ratings_sum = np.array(list(ratings.values())).sum()
    if ratings_sum != 0:
        for i in list(pub_trans_comfort_dist[age_sex_comb].keys()):
            pub_trans_comfort_dist[age_sex_comb][i] = pub_trans_comfort_dist[age_sex_comb][i] / ratings_sum

In [177]:
pub_trans_comfort_dist

{'16-19_K': {0: 0.011904761904761904,
  1: 0.047619047619047616,
  2: 0.2916666666666667,
  3: 0.5059523809523809,
  4: 0.14285714285714285},
 '16-19_M': {0: 0.003968253968253968,
  1: 0.051587301587301584,
  2: 0.30158730158730157,
  3: 0.5515873015873016,
  4: 0.09126984126984126},
 '20-24_K': {0: 0.0055147058823529415,
  1: 0.04779411764705882,
  2: 0.23897058823529413,
  3: 0.47058823529411764,
  4: 0.23713235294117646},
 '20-24_M': {0: 0.008264462809917356,
  1: 0.04132231404958678,
  2: 0.3037190082644628,
  3: 0.493801652892562,
  4: 0.15289256198347106},
 '25-44_K': {0: 0.004651162790697674,
  1: 0.0377906976744186,
  2: 0.45058139534883723,
  3: 0.37732558139534883,
  4: 0.1296511627906977},
 '25-44_M': {0: 0.0054832076764907475,
  1: 0.03015764222069911,
  2: 0.566826593557231,
  3: 0.31802604523646333,
  4: 0.07950651130911583},
 '45-60_K': {0: 0.00995850622406639,
  1: 0.043153526970954356,
  2: 0.38423236514522824,
  3: 0.3991701244813278,
  4: 0.16348547717842324},
 '45-6

### Save probability distribution to .json

In [178]:
out_dir = '../../data/processed/mc/decision_tree'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [179]:
out_file = 'pub_trans_comfort_dist.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(pub_trans_comfort_dist, f)

## Distribution: rating the punctuality of public transport grouped by combinations age group + gender

### Prepare data

In [180]:
data_grouped = data.groupby(['age_sex_comb', 'Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?'])['Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?'].count().reset_index(name='count')

data_grouped.shape[0]

58

In [181]:
data_grouped.head()

,age_sex_comb,Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,count
0,16-19_K,0,6
1,16-19_K,1,7
2,16-19_K,2,52
3,16-19_K,3,89
4,16-19_K,4,14


In [182]:
data[
    (data['age_sex_comb'] == '16-19_K') &
    (data['Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?'] == 2)
].shape[0]

52

### Distribution

In [183]:
unique_age_sex_comb = data_grouped['age_sex_comb'].unique()
max_rating = data_grouped['Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?'].max()

pub_trans_punctuality_dist = {}

for age_sex_comb in unique_age_sex_comb:
    pub_trans_punctuality_dist[age_sex_comb] = {}

    for rating in range(0, max_rating+1):
        try:
            count = int(data_grouped[
                (data_grouped['Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?'] == rating) & 
                (data_grouped['age_sex_comb'] == age_sex_comb)
            ]['count'])
        except TypeError:
            count = 1

        pub_trans_punctuality_dist[age_sex_comb][rating] = count

In [184]:
pub_trans_punctuality_dist

{'16-19_K': {0: 6, 1: 7, 2: 52, 3: 89, 4: 14},
 '16-19_M': {0: 1, 1: 40, 2: 78, 3: 107, 4: 26},
 '20-24_K': {0: 14, 1: 57, 2: 166, 3: 217, 4: 90},
 '20-24_M': {0: 9, 1: 34, 2: 175, 3: 210, 4: 56},
 '25-44_K': {0: 31, 1: 146, 2: 794, 3: 587, 4: 162},
 '25-44_M': {0: 22, 1: 73, 2: 840, 3: 404, 4: 120},
 '45-60_K': {0: 17, 1: 99, 2: 453, 3: 457, 4: 179},
 '45-65_M': {0: 14, 1: 61, 2: 648, 3: 360, 4: 108},
 '6-15_K': {0: 10, 1: 28, 2: 247, 3: 193, 4: 63},
 '6-15_M': {0: 1, 1: 17, 2: 189, 3: 156, 4: 56},
 '61-x_K': {0: 11, 1: 66, 2: 385, 3: 639, 4: 280},
 '66-x_M': {0: 4, 1: 24, 2: 283, 3: 291, 4: 110}}

### Probability distribution

In [185]:
for age_sex_comb, ratings in pub_trans_punctuality_dist.items():
    ratings_sum = np.array(list(ratings.values())).sum()
    if ratings_sum != 0:
        for i in list(pub_trans_punctuality_dist[age_sex_comb].keys()):
            pub_trans_punctuality_dist[age_sex_comb][i] = pub_trans_punctuality_dist[age_sex_comb][i] / ratings_sum

In [186]:
pub_trans_punctuality_dist

{'16-19_K': {0: 0.03571428571428571,
  1: 0.041666666666666664,
  2: 0.30952380952380953,
  3: 0.5297619047619048,
  4: 0.08333333333333333},
 '16-19_M': {0: 0.003968253968253968,
  1: 0.15873015873015872,
  2: 0.30952380952380953,
  3: 0.4246031746031746,
  4: 0.10317460317460317},
 '20-24_K': {0: 0.025735294117647058,
  1: 0.10477941176470588,
  2: 0.30514705882352944,
  3: 0.39889705882352944,
  4: 0.16544117647058823},
 '20-24_M': {0: 0.01859504132231405,
  1: 0.07024793388429752,
  2: 0.3615702479338843,
  3: 0.43388429752066116,
  4: 0.11570247933884298},
 '25-44_K': {0: 0.01802325581395349,
  1: 0.08488372093023255,
  2: 0.46162790697674416,
  3: 0.3412790697674419,
  4: 0.0941860465116279},
 '25-44_M': {0: 0.015078821110349555,
  1: 0.050034270047978065,
  2: 0.5757368060315284,
  3: 0.27690198766278273,
  4: 0.0822481151473612},
 '45-60_K': {0: 0.014107883817427386,
  1: 0.08215767634854772,
  2: 0.3759336099585062,
  3: 0.379253112033195,
  4: 0.14854771784232365},
 '45-65_M'

### Save probability distribution to .json

In [187]:
out_dir = '../../data/processed/mc/decision_tree'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [188]:
out_file = 'pub_trans_punctuality_dist.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(pub_trans_punctuality_dist, f)

## Distribution: rating the comfort of bicycle infrastructure grouped by combinations age group + gender

### Prepare data

In [189]:
data_grouped = data.groupby(['age_sex_comb', 'Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?'])['Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?'].count().reset_index(name='count')

data_grouped.shape[0]

58

In [190]:
data_grouped.head()

,age_sex_comb,Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,count
0,16-19_K,0,2
1,16-19_K,1,4
2,16-19_K,2,48
3,16-19_K,3,70
4,16-19_K,4,44


In [191]:
data[
    (data['age_sex_comb'] == '16-19_K') &
    (data['Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?'] == 2)
].shape[0]

48

### Distribution

In [192]:
unique_age_sex_comb = data_grouped['age_sex_comb'].unique()
max_rating = data_grouped['Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?'].max()

bicycle_infrastr_comfort_dist = {}

for age_sex_comb in unique_age_sex_comb:
    bicycle_infrastr_comfort_dist[age_sex_comb] = {}

    for rating in range(0, max_rating+1):
        try:
            count = int(data_grouped[
                (data_grouped['Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?'] == rating) & 
                (data_grouped['age_sex_comb'] == age_sex_comb)
            ]['count'])
        except TypeError:
            count = 1

        bicycle_infrastr_comfort_dist[age_sex_comb][rating] = count

In [193]:
bicycle_infrastr_comfort_dist

{'16-19_K': {0: 2, 1: 4, 2: 48, 3: 70, 4: 44},
 '16-19_M': {0: 1, 1: 20, 2: 73, 3: 123, 4: 35},
 '20-24_K': {0: 8, 1: 25, 2: 114, 3: 231, 4: 166},
 '20-24_M': {0: 1, 1: 20, 2: 140, 3: 182, 4: 142},
 '25-44_K': {0: 29, 1: 59, 2: 595, 3: 674, 4: 363},
 '25-44_M': {0: 8, 1: 59, 2: 536, 3: 547, 4: 309},
 '45-60_K': {0: 30, 1: 58, 2: 459, 3: 421, 4: 237},
 '45-65_M': {0: 20, 1: 45, 2: 531, 3: 421, 4: 174},
 '6-15_K': {0: 2, 1: 16, 2: 215, 3: 196, 4: 112},
 '6-15_M': {0: 2, 1: 3, 2: 152, 3: 181, 4: 80},
 '61-x_K': {0: 26, 1: 28, 2: 795, 3: 329, 4: 203},
 '66-x_M': {0: 18, 1: 26, 2: 375, 3: 172, 4: 121}}

### Probability distribution

In [194]:
for age_sex_comb, ratings in bicycle_infrastr_comfort_dist.items():
    ratings_sum = np.array(list(ratings.values())).sum()
    if ratings_sum != 0:
        for i in list(bicycle_infrastr_comfort_dist[age_sex_comb].keys()):
            bicycle_infrastr_comfort_dist[age_sex_comb][i] = bicycle_infrastr_comfort_dist[age_sex_comb][i] / ratings_sum

In [195]:
bicycle_infrastr_comfort_dist

{'16-19_K': {0: 0.011904761904761904,
  1: 0.023809523809523808,
  2: 0.2857142857142857,
  3: 0.4166666666666667,
  4: 0.2619047619047619},
 '16-19_M': {0: 0.003968253968253968,
  1: 0.07936507936507936,
  2: 0.2896825396825397,
  3: 0.4880952380952381,
  4: 0.1388888888888889},
 '20-24_K': {0: 0.014705882352941176,
  1: 0.04595588235294118,
  2: 0.20955882352941177,
  3: 0.42463235294117646,
  4: 0.30514705882352944},
 '20-24_M': {0: 0.002061855670103093,
  1: 0.041237113402061855,
  2: 0.28865979381443296,
  3: 0.3752577319587629,
  4: 0.2927835051546392},
 '25-44_K': {0: 0.01686046511627907,
  1: 0.03430232558139535,
  2: 0.34593023255813954,
  3: 0.3918604651162791,
  4: 0.21104651162790697},
 '25-44_M': {0: 0.0054832076764907475,
  1: 0.04043865661411926,
  2: 0.36737491432488006,
  3: 0.37491432488005483,
  4: 0.2117888965044551},
 '45-60_K': {0: 0.024896265560165973,
  1: 0.048132780082987554,
  2: 0.3809128630705394,
  3: 0.34937759336099583,
  4: 0.1966804979253112},
 '45-65_

### Save probability distribution to .json

In [196]:
out_dir = '../../data/processed/mc/decision_tree'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [197]:
out_file = 'bicycle_infrastr_comfort_dist.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(bicycle_infrastr_comfort_dist, f)

## Distribution: pedestrian inconvenience grouped by combinations age group + gender

### Prepare data

In [198]:
data['PIESZO-niedogodności'] = 0
for col in data.columns:
    if col.startswith('PIESZO '):
        data['PIESZO-niedogodności'] += data[col]

In [199]:
data_grouped = data.groupby(['age_sex_comb', 'PIESZO-niedogodności'])['PIESZO-niedogodności'].count().reset_index(name='count')

data_grouped.shape[0]

48

In [200]:
data_grouped.head()

,age_sex_comb,PIESZO-niedogodności,count
0,16-19_K,0,52
1,16-19_K,1,40
2,16-19_K,2,39
3,16-19_K,3,37
4,16-19_M,0,78


In [201]:
data[
    (data['age_sex_comb'] == '16-19_K') &
    (data['PIESZO-niedogodności'] == 2)
].shape[0]

39

### Distribution

In [202]:
unique_age_sex_comb = data_grouped['age_sex_comb'].unique()
max_inconveniences = data_grouped['PIESZO-niedogodności'].max()

pedestrian_inconvenience_dist = {}

for age_sex_comb in unique_age_sex_comb:
    pedestrian_inconvenience_dist[age_sex_comb] = {}

    for inconveniences in range(0, max_inconveniences+1):
        try:
            count = int(data_grouped[
                (data_grouped['PIESZO-niedogodności'] == inconveniences) & 
                (data_grouped['age_sex_comb'] == age_sex_comb)
            ]['count'])
        except TypeError:
            count = 1

        pedestrian_inconvenience_dist[age_sex_comb][inconveniences] = count

In [203]:
pedestrian_inconvenience_dist

{'16-19_K': {0: 52, 1: 40, 2: 39, 3: 37},
 '16-19_M': {0: 78, 1: 49, 2: 71, 3: 53},
 '20-24_K': {0: 152, 1: 118, 2: 152, 3: 122},
 '20-24_M': {0: 167, 1: 108, 2: 98, 3: 111},
 '25-44_K': {0: 426, 1: 395, 2: 459, 3: 440},
 '25-44_M': {0: 425, 1: 346, 2: 367, 3: 321},
 '45-60_K': {0: 267, 1: 254, 2: 294, 3: 390},
 '45-65_M': {0: 285, 1: 274, 2: 284, 3: 348},
 '6-15_K': {0: 215, 1: 71, 2: 128, 3: 127},
 '6-15_M': {0: 144, 1: 105, 2: 75, 3: 94},
 '61-x_K': {0: 273, 1: 316, 2: 364, 3: 428},
 '66-x_M': {0: 168, 1: 161, 2: 173, 3: 210}}

### Probability distribution

In [204]:
for age_sex_comb, inconveniences in pedestrian_inconvenience_dist.items():
    inconveniences_sum = np.array(list(inconveniences.values())).sum()
    if inconveniences_sum != 0:
        for i in list(pedestrian_inconvenience_dist[age_sex_comb].keys()):
            pedestrian_inconvenience_dist[age_sex_comb][i] = pedestrian_inconvenience_dist[age_sex_comb][i] / inconveniences_sum

In [205]:
pedestrian_inconvenience_dist

{'16-19_K': {0: 0.30952380952380953,
  1: 0.23809523809523808,
  2: 0.23214285714285715,
  3: 0.22023809523809523},
 '16-19_M': {0: 0.3107569721115538,
  1: 0.1952191235059761,
  2: 0.28286852589641437,
  3: 0.21115537848605578},
 '20-24_K': {0: 0.27941176470588236,
  1: 0.21691176470588236,
  2: 0.27941176470588236,
  3: 0.22426470588235295},
 '20-24_M': {0: 0.3450413223140496,
  1: 0.2231404958677686,
  2: 0.2024793388429752,
  3: 0.22933884297520662},
 '25-44_K': {0: 0.24767441860465117,
  1: 0.22965116279069767,
  2: 0.2668604651162791,
  3: 0.2558139534883721},
 '25-44_M': {0: 0.29129540781357094,
  1: 0.2371487320082248,
  2: 0.25154215215901304,
  3: 0.22001370801919123},
 '45-60_K': {0: 0.22157676348547717,
  1: 0.2107883817427386,
  2: 0.24398340248962655,
  3: 0.3236514522821577},
 '45-65_M': {0: 0.23929471032745592,
  1: 0.23005877413937867,
  2: 0.23845507976490343,
  3: 0.29219143576826195},
 '6-15_K': {0: 0.3974121996303142,
  1: 0.13123844731977818,
  2: 0.23659889094269

### Save probability distribution to .json

In [206]:
out_dir = '../../data/processed/mc/decision_tree'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [207]:
out_file = 'pedestrian_inconvenience_dist.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(pedestrian_inconvenience_dist, f)

## Distribution: household persons grouped by combinations age group + gender

### Prepare data

In [219]:
data_grouped = data.groupby(['age_sex_comb', 'Liczba osób w gospodarstwie domowym [ogółem]'])['Liczba osób w gospodarstwie domowym [ogółem]'].count().reset_index(name='count')

data_grouped.shape[0]

96

In [221]:
data_grouped['Liczba osób w gospodarstwie domowym [ogółem]'] = data_grouped['Liczba osób w gospodarstwie domowym [ogółem]'].astype('int')

In [222]:
data_grouped.head()

,age_sex_comb,Liczba osób w gospodarstwie domowym [ogółem],count
0,16-19_K,1,18
1,16-19_K,2,10
2,16-19_K,3,58
3,16-19_K,4,66
4,16-19_K,5,10


In [223]:
data[
    (data['age_sex_comb'] == '16-19_K') &
    (data['Liczba osób w gospodarstwie domowym [ogółem]'] == 2)
].shape[0]

10

### Distribution

In [230]:
unique_age_sex_comb = data_grouped['age_sex_comb'].unique()
max_household_persons = data_grouped['Liczba osób w gospodarstwie domowym [ogółem]'].max()

household_persons_dist = {}

for age_sex_comb in unique_age_sex_comb:
    household_persons_dist[age_sex_comb] = {}

    for household_persons in range(1, max_household_persons+1):
        try:
            count = int(data_grouped[
                (data_grouped['Liczba osób w gospodarstwie domowym [ogółem]'] == household_persons) & 
                (data_grouped['age_sex_comb'] == age_sex_comb)
            ]['count'])
        except TypeError:
            count = 1

        household_persons_dist[age_sex_comb][household_persons] = count

In [234]:
for age_sex_comb, household_persons in household_persons_dist.items():
    print(age_sex_comb, end='\t -  ')
    print(household_persons)
    
# household_persons_dist

16-19_K	 -  {1: 18, 2: 10, 3: 58, 4: 66, 5: 10, 6: 4, 7: 2, 8: 1, 9: 1, 10: 1}
16-19_M	 -  {1: 20, 2: 19, 3: 93, 4: 83, 5: 19, 6: 14, 7: 2, 8: 1, 9: 1, 10: 1}
20-24_K	 -  {1: 134, 2: 135, 3: 140, 4: 84, 5: 21, 6: 12, 7: 18, 8: 1, 9: 1, 10: 1}
20-24_M	 -  {1: 151, 2: 120, 3: 110, 4: 77, 5: 11, 6: 15, 7: 1, 8: 1, 9: 1, 10: 1}
25-44_K	 -  {1: 140, 2: 427, 3: 560, 4: 414, 5: 110, 6: 50, 7: 15, 8: 4, 9: 1, 10: 1}
25-44_M	 -  {1: 157, 2: 394, 3: 446, 4: 321, 5: 95, 6: 20, 7: 18, 8: 6, 9: 2, 10: 1}
45-60_K	 -  {1: 98, 2: 373, 3: 369, 4: 255, 5: 54, 6: 30, 7: 14, 8: 6, 9: 2, 10: 4}
45-65_M	 -  {1: 101, 2: 378, 3: 394, 4: 227, 5: 45, 6: 37, 7: 6, 8: 3, 9: 1, 10: 1}
6-15_K	 -  {1: 1, 2: 24, 3: 162, 4: 228, 5: 70, 6: 41, 7: 8, 8: 4, 9: 1, 10: 4}
6-15_M	 -  {1: 1, 2: 24, 3: 111, 4: 172, 5: 62, 6: 33, 7: 10, 8: 1, 9: 2, 10: 4}
61-x_K	 -  {1: 366, 2: 650, 3: 187, 4: 110, 5: 39, 6: 17, 7: 8, 8: 4, 9: 1, 10: 1}
66-x_M	 -  {1: 129, 2: 418, 3: 99, 4: 34, 5: 12, 6: 12, 7: 2, 8: 4, 9: 2, 10: 1}


### Probability distribution

In [235]:
for age_sex_comb, household_persons in household_persons_dist.items():
    household_persons_sum = np.array(list(household_persons.values())).sum()
    if household_persons_sum != 0:
        for i in list(household_persons_dist[age_sex_comb].keys()):
            household_persons_dist[age_sex_comb][i] = household_persons_dist[age_sex_comb][i] / household_persons_sum

In [236]:
for age_sex_comb, household_persons in household_persons_dist.items():
    print(age_sex_comb, end='\t -  ')
    print(household_persons)
    
# household_persons_dist

16-19_K	 -  {1: 0.10526315789473684, 2: 0.05847953216374269, 3: 0.3391812865497076, 4: 0.38596491228070173, 5: 0.05847953216374269, 6: 0.023391812865497075, 7: 0.011695906432748537, 8: 0.005847953216374269, 9: 0.005847953216374269, 10: 0.005847953216374269}
16-19_M	 -  {1: 0.07905138339920949, 2: 0.07509881422924901, 3: 0.3675889328063241, 4: 0.32806324110671936, 5: 0.07509881422924901, 6: 0.05533596837944664, 7: 0.007905138339920948, 8: 0.003952569169960474, 9: 0.003952569169960474, 10: 0.003952569169960474}
20-24_K	 -  {1: 0.2449725776965265, 2: 0.24680073126142596, 3: 0.25594149908592323, 4: 0.15356489945155394, 5: 0.038391224862888484, 6: 0.021937842778793418, 7: 0.03290676416819013, 8: 0.0018281535648994515, 9: 0.0018281535648994515, 10: 0.0018281535648994515}
20-24_M	 -  {1: 0.3094262295081967, 2: 0.2459016393442623, 3: 0.22540983606557377, 4: 0.15778688524590165, 5: 0.022540983606557378, 6: 0.030737704918032786, 7: 0.0020491803278688526, 8: 0.0020491803278688526, 9: 0.0020491803

### Save probability distribution to .json

In [237]:
out_dir = '../../data/processed/mc/decision_tree'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [238]:
out_file = 'household_persons_dist.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(household_persons_dist, f)

## Distribution: household cars grouped by combinations age group + gender

### Prepare data

In [240]:
data_grouped = data.groupby(['age_sex_comb', 'Samochód'])['Samochód'].count().reset_index(name='count')

data_grouped.shape[0]

71

In [242]:
data_grouped['Samochód'] = data_grouped['Samochód'].astype('int')

In [243]:
data_grouped.head()

,age_sex_comb,Samochód,count
0,16-19_K,0,30
1,16-19_K,1,87
2,16-19_K,2,42
3,16-19_K,3,6
4,16-19_K,4,3


In [245]:
data[
    (data['age_sex_comb'] == '16-19_K') &
    (data['Samochód'] == 2)
].shape[0]

42

### Distribution

In [249]:
unique_age_sex_comb = data_grouped['age_sex_comb'].unique()
max_household_cars = data_grouped['Samochód'].max()

household_cars_dist = {}

for age_sex_comb in unique_age_sex_comb:
    household_cars_dist[age_sex_comb] = {}

    for household_cars in range(0, max_household_cars+1):
        try:
            count = int(data_grouped[
                (data_grouped['Samochód'] == household_cars) & 
                (data_grouped['age_sex_comb'] == age_sex_comb)
            ]['count'])
        except TypeError:
            count = 1

        household_cars_dist[age_sex_comb][household_cars] = count

In [252]:
for age_sex_comb, household_cars in household_cars_dist.items():
    print(age_sex_comb, end='\t -  ')
    print(household_cars)
    
# household_cars_dist

16-19_K	 -  {0: 30, 1: 87, 2: 42, 3: 6, 4: 3, 5: 1, 6: 1, 7: 1}
16-19_M	 -  {0: 49, 1: 107, 2: 89, 3: 2, 4: 4, 5: 1, 6: 1, 7: 1}
20-24_K	 -  {0: 239, 1: 176, 2: 103, 3: 12, 4: 6, 5: 5, 6: 1, 7: 3}
20-24_M	 -  {0: 238, 1: 170, 2: 64, 3: 10, 4: 2, 5: 1, 6: 1, 7: 1}
25-44_K	 -  {0: 349, 1: 848, 2: 449, 3: 43, 4: 22, 5: 3, 6: 1, 7: 6}
25-44_M	 -  {0: 235, 1: 759, 2: 391, 3: 49, 4: 23, 5: 1, 6: 1, 7: 2}
45-60_K	 -  {0: 297, 1: 590, 2: 263, 3: 39, 4: 14, 5: 1, 6: 2, 7: 1}
45-65_M	 -  {0: 199, 1: 673, 2: 259, 3: 39, 4: 19, 5: 2, 6: 1, 7: 1}
6-15_K	 -  {0: 48, 1: 314, 2: 163, 3: 6, 4: 8, 5: 1, 6: 1, 7: 2}
6-15_M	 -  {0: 57, 1: 198, 2: 141, 3: 11, 4: 7, 5: 1, 6: 2, 7: 2}
61-x_K	 -  {0: 720, 1: 515, 2: 122, 3: 16, 4: 8, 5: 1, 6: 1, 7: 1}
66-x_M	 -  {0: 301, 1: 340, 2: 57, 3: 8, 4: 4, 5: 1, 6: 2, 7: 1}


### Probability distribution

In [254]:
for age_sex_comb, household_cars in household_cars_dist.items():
    household_cars_sum = np.array(list(household_cars.values())).sum()
    if household_cars_sum != 0:
        for i in list(household_cars_dist[age_sex_comb].keys()):
            household_cars_dist[age_sex_comb][i] = household_cars_dist[age_sex_comb][i] / household_cars_sum

In [255]:
for age_sex_comb, household_cars in household_cars_dist.items():
    print(age_sex_comb, end='\t -  ')
    print(household_cars)
    
# household_cars_dist

16-19_K	 -  {0: 0.17543859649122806, 1: 0.5087719298245614, 2: 0.24561403508771928, 3: 0.03508771929824561, 4: 0.017543859649122806, 5: 0.005847953216374269, 6: 0.005847953216374269, 7: 0.005847953216374269}
16-19_M	 -  {0: 0.19291338582677164, 1: 0.421259842519685, 2: 0.35039370078740156, 3: 0.007874015748031496, 4: 0.015748031496062992, 5: 0.003937007874015748, 6: 0.003937007874015748, 7: 0.003937007874015748}
20-24_K	 -  {0: 0.43853211009174314, 1: 0.3229357798165138, 2: 0.1889908256880734, 3: 0.022018348623853212, 4: 0.011009174311926606, 5: 0.009174311926605505, 6: 0.001834862385321101, 7: 0.005504587155963303}
20-24_M	 -  {0: 0.4887063655030801, 1: 0.3490759753593429, 2: 0.13141683778234087, 3: 0.02053388090349076, 4: 0.004106776180698152, 5: 0.002053388090349076, 6: 0.002053388090349076, 7: 0.002053388090349076}
25-44_K	 -  {0: 0.20278907611853572, 1: 0.4927367809413132, 2: 0.2608948285880302, 3: 0.024985473561882625, 4: 0.012783265543288786, 5: 0.0017431725740848344, 6: 0.00058

### Save probability distribution to .json

In [256]:
out_dir = '../../data/processed/mc/decision_tree'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [257]:
out_file = 'household_cars_dist.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(household_cars_dist, f)

## Distribution: household bicycles grouped by combinations age group + gender

### Prepare data

In [259]:
data_grouped = data.groupby(['age_sex_comb', 'Rower'])['Rower'].count().reset_index(name='count')

data_grouped.shape[0]

96

In [261]:
data_grouped['Rower'] = data_grouped['Rower'].astype('int')

In [262]:
data_grouped.head()

,age_sex_comb,Rower,count
0,16-19_K,0,41
1,16-19_K,1,17
2,16-19_K,2,39
3,16-19_K,3,38
4,16-19_K,4,29


In [263]:
data[
    (data['age_sex_comb'] == '16-19_K') &
    (data['Rower'] == 2)
].shape[0]

39

### Distribution

In [269]:
unique_age_sex_comb = data_grouped['age_sex_comb'].unique()
max_household_bicycles = data_grouped['Rower'].max()

household_bicycles_dist = {}

for age_sex_comb in unique_age_sex_comb:
    household_bicycles_dist[age_sex_comb] = {}

    for household_bicycles in range(0, max_household_bicycles+1):
        try:
            count = int(data_grouped[
                (data_grouped['Rower'] == household_bicycles) & 
                (data_grouped['age_sex_comb'] == age_sex_comb)
            ]['count'])
        except TypeError:
            count = 1

        household_bicycles_dist[age_sex_comb][household_bicycles] = count

In [270]:
for age_sex_comb, household_bicycles in household_bicycles_dist.items():
    print(age_sex_comb, end='\t -  ')
    print(household_bicycles)
    
# household_bicycles_dist

16-19_K	 -  {0: 41, 1: 17, 2: 39, 3: 38, 4: 29, 5: 2, 6: 1, 7: 1, 8: 2, 9: 1, 10: 1}
16-19_M	 -  {0: 46, 1: 38, 2: 84, 3: 48, 4: 29, 5: 2, 6: 2, 7: 1, 8: 2, 9: 1, 10: 1}
20-24_K	 -  {0: 235, 1: 120, 2: 88, 3: 60, 4: 26, 5: 2, 6: 6, 7: 4, 8: 3, 9: 1, 10: 1}
20-24_M	 -  {0: 207, 1: 123, 2: 63, 3: 44, 4: 43, 5: 2, 6: 1, 7: 1, 8: 2, 9: 1, 10: 1}
25-44_K	 -  {0: 424, 1: 359, 2: 477, 3: 248, 4: 160, 5: 29, 6: 13, 7: 4, 8: 6, 9: 1, 10: 1}
25-44_M	 -  {0: 349, 1: 338, 2: 431, 3: 206, 4: 99, 5: 25, 6: 9, 7: 2, 8: 1, 9: 1, 10: 1}
45-60_K	 -  {0: 356, 1: 242, 2: 307, 3: 161, 4: 104, 5: 19, 6: 3, 7: 2, 8: 4, 9: 1, 10: 7}
45-65_M	 -  {0: 326, 1: 271, 2: 313, 3: 162, 4: 86, 5: 12, 6: 9, 7: 4, 8: 8, 9: 1, 10: 1}
6-15_K	 -  {0: 58, 1: 84, 2: 125, 3: 132, 4: 124, 5: 16, 6: 1, 7: 1, 8: 1, 9: 1, 10: 2}
6-15_M	 -  {0: 73, 1: 35, 2: 104, 3: 83, 4: 100, 5: 18, 6: 1, 7: 1, 8: 5, 9: 1, 10: 1}
61-x_K	 -  {0: 771, 1: 268, 2: 256, 3: 59, 4: 19, 5: 4, 6: 2, 7: 1, 8: 2, 9: 1, 10: 1}
66-x_M	 -  {0: 395, 1: 125, 2: 

### Probability distribution

In [271]:
for age_sex_comb, household_bicycles in household_bicycles_dist.items():
    household_bicycles_sum = np.array(list(household_bicycles.values())).sum()
    if household_bicycles_sum != 0:
        for i in list(household_bicycles_dist[age_sex_comb].keys()):
            household_bicycles_dist[age_sex_comb][i] = household_bicycles_dist[age_sex_comb][i] / household_bicycles_sum

In [272]:
for age_sex_comb, household_bicycles in household_bicycles_dist.items():
    print(age_sex_comb, end='\t -  ')
    print(household_bicycles)
    
# household_bicycles_dist

16-19_K	 -  {0: 0.23837209302325582, 1: 0.09883720930232558, 2: 0.22674418604651161, 3: 0.22093023255813954, 4: 0.1686046511627907, 5: 0.011627906976744186, 6: 0.005813953488372093, 7: 0.005813953488372093, 8: 0.011627906976744186, 9: 0.005813953488372093, 10: 0.005813953488372093}
16-19_M	 -  {0: 0.18110236220472442, 1: 0.14960629921259844, 2: 0.33070866141732286, 3: 0.1889763779527559, 4: 0.1141732283464567, 5: 0.007874015748031496, 6: 0.007874015748031496, 7: 0.003937007874015748, 8: 0.007874015748031496, 9: 0.003937007874015748, 10: 0.003937007874015748}
20-24_K	 -  {0: 0.43040293040293043, 1: 0.21978021978021978, 2: 0.16117216117216118, 3: 0.10989010989010989, 4: 0.047619047619047616, 5: 0.003663003663003663, 6: 0.01098901098901099, 7: 0.007326007326007326, 8: 0.005494505494505495, 9: 0.0018315018315018315, 10: 0.0018315018315018315}
20-24_M	 -  {0: 0.42418032786885246, 1: 0.2520491803278688, 2: 0.1290983606557377, 3: 0.09016393442622951, 4: 0.08811475409836066, 5: 0.0040983606557

### Save probability distribution to .json

In [273]:
out_dir = '../../data/processed/mc/decision_tree'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [274]:
out_file = 'household_bicycles_dist.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(household_bicycles_dist, f)